# Test get images sizes and add them to the sheet
Test image-sizes-parser with less data

In [86]:
import pandas as pd
import asyncio
from io import BytesIO
from datetime import datetime

import aiohttp
from PIL import Image

In [87]:
SHEET_ID = "1QX2IhFyYmGDFMvovw2WFz3wAT4piAZ_8hi5Lzp7LjV0"
SHEET_NAME = "feed"
NEW_SHEET_FILE = "TestParserImageSize.xlsx"
COLUMN_NAME = "SIZE"

In [88]:
start = datetime.now()

Get data from sheet (300 rows for test)

In [89]:
sheet_url = (
    f"https://docs.google.com/spreadsheets/d/"
    f"{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}"
)
dataframe = pd.read_csv(sheet_url)[:300]
dataframe.head()

,image_url,SIZE
0,https://data.sanitino.eu/PRODUCT-33916/8607663...,575x860
1,https://data.sanitino.eu/PRODUCT-62434/f7aa3c2...,NaN
2,https://data.sanitino.eu/PRODUCT-33915/8607663...,NaN
3,https://data.sanitino.eu/PRODUCT-62426/506d67b...,NaN
4,https://data.sanitino.eu/PRODUCT-33893/ccd99b4...,NaN


In [99]:
image_urls_dataframe = dataframe["image_url"]
dataframe_size = image_urls_dataframe.size
dataframe_size

300

Divide the urls into dictionaries (packs) <= 100 each & get image sizes
(In the main parser the urls divides into packs <= 10_000 each)

In [100]:
async def get_size(row: int, url: str, session: aiohttp.ClientSession):
    """The coroutine gets the image size from the url"""
    if type(url) != str:
        return {row: None}

    async with session.get(url) as response:
        if response.status != 200:
            return {row: "ImageNotFound"}

        content = await response.content.read()
        with Image.open(BytesIO(content)) as image:
            image_size = image.size

    image_size = f"{image_size[0]}x{image_size[1]}"

    return {row: image_size}


async def get_images_sizes(image_urls_: dict):
    """Create an async client and event loop"""
    async with aiohttp.ClientSession() as session:
        return await asyncio.gather(
            *[
                get_size(image_row, image_url, session)
                for image_row, image_url in image_urls_.items()
            ]
        )

pack_size = 100
packs_amount = dataframe_size // pack_size + 1

all_sizes_list = []
for i in range(packs_amount):
    urls_pack = image_urls_dataframe[pack_size * i:pack_size * (i + 1)].to_dict()
    image_sizes = await get_images_sizes(urls_pack)
    all_sizes_list += image_sizes

all_sizes_list[:5]

[{0: '1080x1614'},
 {1: '1080x1080'},
 {2: '1080x1614'},
 {3: '1080x1080'},
 {4: '1080x1614'}]

In [101]:
all_sizes = {}

for dictionary in all_sizes_list:
    all_sizes.update(dictionary)

Update sheet size column with new data

In [103]:
data = dataframe
sizes_list = list(all_sizes.values())
new_data = pd.DataFrame({COLUMN_NAME: sizes_list})
data.update(new_data)

with pd.ExcelWriter(NEW_SHEET_FILE, engine="openpyxl") as writer:
    data.to_excel(writer, sheet_name=SHEET_NAME, index=False)

new_data.head()

In [104]:
finish = datetime.now()

In [105]:
print(
    f"Start time: {start.strftime('%H:%M:%S')}\n"
    f"Finish time: {finish.strftime('%H:%M:%S')}\n"
    f"Execution: {(finish - start).total_seconds()} sec"
)

Start time: 13:45:06
Finish time: 13:48:10
Execution: 183.509687 sec
